In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
TUIK Harita JSON -> CSV/Excel Dönüştürücü
Kullanım:
  python tuik_json_to_excel.py --dir ./jsonler --out ./out
  python tuik_json_to_excel.py --urls ./urls.txt --out ./out --save-raw
"""
import argparse, json, os, sys, time, pathlib
from typing import List, Dict, Any
try:
    import requests
except Exception:
    requests = None
import pandas as pd

PLAKA_TO_IL = {1:"Adana",2:"Adıyaman",3:"Afyonkarahisar",4:"Ağrı",5:"Amasya",6:"Ankara",7:"Antalya",8:"Artvin",9:"Aydın",
10:"Balıkesir",11:"Bilecik",12:"Bingöl",13:"Bitlis",14:"Bolu",15:"Burdur",16:"Bursa",17:"Çanakkale",18:"Çankırı",
19:"Çorum",20:"Denizli",21:"Diyarbakır",22:"Edirne",23:"Elazığ",24:"Erzincan",25:"Erzurum",26:"Eskişehir",
27:"Gaziantep",28:"Giresun",29:"Gümüşhane",30:"Hakkâri",31:"Hatay",32:"Isparta",33:"Mersin",34:"İstanbul",
35:"İzmir",36:"Kars",37:"Kastamonu",38:"Kayseri",39:"Kırklareli",40:"Kırşehir",41:"Kocaeli",42:"Konya",
43:"Kütahya",44:"Malatya",45:"Manisa",46:"Kahramanmaraş",47:"Mardin",48:"Muğla",49:"Muş",50:"Nevşehir",
51:"Niğde",52:"Ordu",53:"Rize",54:"Sakarya",55:"Samsun",56:"Siirt",57:"Sinop",58:"Sivas",59:"Tekirdağ",
60:"Tokat",61:"Trabzon",62:"Tunceli",63:"Şanlıurfa",64:"Uşak",65:"Van",66:"Yozgat",67:"Zonguldak",68:"Aksaray",
69:"Bayburt",70:"Karaman",71:"Kırıkkale",72:"Batman",73:"Şırnak",74:"Bartın",75:"Ardahan",76:"Iğdır",
77:"Yalova",78:"Karabük",79:"Kilis",80:"Osmaniye",81:"Düzce"}

def read_json_file(path: pathlib.Path) -> Dict[str, Any]:
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)

def fetch_json(url: str, timeout: int = 30, retries: int = 2, sleep: float = 1.0) -> Dict[str, Any]:
    if requests is None:
        raise RuntimeError("requests yok. `pip install requests`")
    last_exc = None
    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/json,text/plain,*/*"}
    for _ in range(retries + 1):
        try:
            r = requests.get(url, headers=headers, timeout=timeout)
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last_exc = e
            time.sleep(sleep)
    raise last_exc

def parse_tuik_json(d: Dict[str, Any], source: str) -> pd.DataFrame:
    years = [int(y) for y in d.get("tarihler", [])]
    rows = []
    for item in d.get("veriler", []):
        code = int(item["duzeyKodu"])
        il = PLAKA_TO_IL.get(code, f"Bilinmeyen-{code}")
        values = [int(v) for v in item.get("veri", [])]
        for year, value in zip(years, values):
            rows.append({
                "il_kodu": code, "il": il, "yil": year, "nufus": value,
                "kaynak": source, "gostergeNo": d.get("gostergeNo"),
                "gosterge_ad": d.get("gosterge_ad"), "period": d.get("period"),
            })
    return pd.DataFrame(rows)

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--dir", type=str)
    ap.add_argument("--urls", type=str)
    ap.add_argument("--out", type=str, required=True)
    ap.add_argument("--save-raw", action="store_true")
    ap.add_argument("--timeout", type=int, default=30)
    args = ap.parse_args()

    outdir = pathlib.Path(args.out); outdir.mkdir(parents=True, exist_ok=True)
    rawdir = outdir/"raw";  rawdir.mkdir(parents=True, exist_ok=True) if args.save_raw else None

    frames: List[pd.DataFrame] = []

    if args.dir:
        p = pathlib.Path(args.dir)
        files = sorted(p.glob("*.json")) if p.exists() else []
        for fp in files:
            try:
                frames.append(parse_tuik_json(read_json_file(fp), source=str(fp)))
            except Exception as e:
                print(f"Hata (dosya: {fp}): {e}", file=sys.stderr)

    if args.urls:
        urls_path = pathlib.Path(args.urls)
        urls = [ln.strip() for ln in urls_path.read_text(encoding="utf-8").splitlines()
                if ln.strip() and not ln.strip().startswith("#")] if urls_path.exists() else []
        for u in urls:
            try:
                d = fetch_json(u, timeout=args.timeout)
                if args.save_raw:
                    safe = u.replace('://','_').replace('/','_').replace('?','_').replace('&','_').replace('=','_')
                    (rawdir/f"{safe}.json").write_text(json.dumps(d, ensure_ascii=False, indent=2), encoding="utf-8")
                frames.append(parse_tuik_json(d, source=u))
            except Exception as e:
                print(f"Hata (URL: {u}): {e}", file=sys.stderr)

    if not frames:
        print("Hiç veri toplanamadı.", file=sys.stderr); sys.exit(2)

    long_df = pd.concat(frames, ignore_index=True)
    long_df.sort_values(["il_kodu","yil","kaynak"], ascending=[True, False, True], inplace=True)
    long_csv = outdir/"nufus_birlesik_uzun.csv"; long_df.to_csv(long_csv, index=False, encoding="utf-8")

    latest = long_df.drop_duplicates(subset=["il_kodu","il","yil"], keep="first")
    wide_df = latest.pivot_table(index=["il_kodu","il"], columns="yil", values="nufus").reset_index()
    wide_df.columns.name = None
    wide_csv = outdir/"nufus_birlesik_genis.csv"; wide_df.to_csv(wide_csv, index=False, encoding="utf-8")

    xlsx_path = outdir/"nufus_birlesik.xlsx"
    with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as w:
        long_df.to_excel(w, sheet_name="Uzun_Form", index=False)
        wide_df.to_excel(w, sheet_name="Genis_Form", index=False)
    print(f"OK\nCSV (uzun): {long_csv}\nCSV (geniş): {wide_csv}\nExcel: {xlsx_path}")

if __name__ == "__main__":
    main()
